# Distributed training of a TensorFlow 2.x model with custom training loop on the Amazon SageMaker optimized TensorFlow container using the Amazon SageMaker Distributed Data Parallel library and debug using Amazon SageMaker Debugger


***본 노트북 코드는 [영문 노트북](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-featurestore/sagemaker_featurestore_fraud_detection_python_sdk.ipynb)의 번역본으로 직역이 아닌 중간중간 설명을 덧붙이고 코드 오류를 수정했습니다.***

[Amazon SageMaker](https://aws.amazon.com/sagemaker/)는 완전 관리형 머신 러닝 서비스입니다. SageMaker를 사용하면 기본 제공 알고리즘을 사용하고 자체 알고리즘 및 프레임워크를 가져올 수 있습니다. 그러한 프레임워크 중 하나가 TensorFlow 2.x입니다. 이 프레임워크로 분산 훈련을 수행할 때 SageMaker의 [Distributed Data Parallel](https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel.html) 또는 [Distributed Model Parallel](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel.html) 라이브러리를 사용할 수 있습니다. [Amazon SageMaker Debugger](https://docs.aws.amazon.com/sagemaker/latest/dg/train-debugger.html)는 훈련 작업을 실시간으로 디버그, 모니터링 및 프로파일링하여 수렴하지 않는 조건을 감지하고 병목 현상을 제거하여 리소스 사용률을 최적화하며 훈련 시간을 개선하고 머신 러닝 모델의 비용을 절감합니다.

이 노트북은 SageMaker에 최적화된 TensorFlow 2.x 컨테이너를 사용하여 SageMaker 분산 데이터 병렬 라이브러리를 사용하는 [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) 데이터셋을 사용하는 다중 클래스 이미지 분류 모델의 분산 학습을 수행하고 SageMaker 디버거를 사용하여 디버그하는 방법을 보여줍니다. 

**Note:**

* 이 노트북은 SageMaker 기본 API를 참조하므로 SageMaker 노트북 인스턴스 내에서만 실행해야 합니다.
* 이 노트북을 작성할 당시 이 노트북과 가장 관련성이 높은 Jupyter 노트북 커널의 최신 버전은 `conda_tensorflow2_p36` 입니다.
* 이 노트북에는 다중 GPU를 사용하는 인스턴스가 최소 2개 이상 필요합니다. (예: `ml.p3.16xlarge`)
* 이 노트북은 동일한 AWS 계정 및 이 노트북이 실행되는 동일한 리전에 리소스를 생성합니다.

**Table of Contents:**

1. [Complete prerequisites](#Complete%20prerequisites)

    1. [Check and configure access to the Internet](#Check%20and%20configure%20access%20to%20the%20Internet)

    2. [Check and upgrade required software versions](#Check%20and%20upgrade%20required%20software%20versions)
    
    3. [Check and configure security permissions](#Check%20and%20configure%20security%20permissions)

    4. [Organize imports](#Organize%20imports)
    
    5. [Create common objects](#Create%20common%20objects)

2. [Prepare the dataset](#Prepare%20the%20dataset)

    1. [Create the local directories](#Create%20the%20local%20directories)

    2. [Load the dataset](#Load%20the%20dataset)
    
    3. [View the details of the dataset](#View%20the%20details%20of%20the%20dataset)
    
    4. [Visualize the dataset](#Visualize%20the%20dataset)
    
    5. [Normalize the dataset](#Normalize%20the%20dataset)
    
    6. [Save the prepared datasets locally](#Save%20the%20prepared%20datasets%20locally)
    
    7. [Upload the prepared datasets to S3](#Upload%20the%20prepared%20datasets%20to%20S3)

3. [View the training script](#View%20the%20training%20script)

    1. [Zero script change](#Zero%20script%20change)
    
    2. [With script change](#With%20script%20change)

4. [Perform training, validation and testing](#Perform%20training%20validation%20and%20testing)

    1. [Set the training parameters](#Set%20the%20training%20parameters)
    
    2. [Set the debugger parameters](#Set%20the%20debugger%20parameters)
    
    3. [(Optional) Delete previous checkpoints](#(Optional)%20Delete%20previous%20checkpoints)

    4. [Run the training job](#Run%20the%20training%20job)

5. [View the auto-generated debugger profiling report](#View%20the%20auto-generated%20debugger%20profiling%20report)

6. [Perform interactive analysis of the debugger output](#Perform%20interactive%20analysis%20of%20the%20debugger%20output)

    1. [Get the training job](#Get%20the%20training%20job)

    2. [Read the metrics](#Read%20the%20metrics)

    3. [Plot the metrics](#Plot%20the%20metrics)
    
        1. [System metrics histogram](#System%20metrics%20histogram)

        2. [Framework metrics stepline chart](#Framework%20metrics%20stepline%20chart)
        
        3. [Framework metrics step histogram](#Framework%20metrics%20step%20histogram)

        4. [System and framework metrics timeline charts](#System%20and%20framework%20metrics%20timeline%20charts)

        5. [System and framework metrics heatmap](#System%20and%20framework%20metrics%20heatmap)
        

7. [Cleanup](#Cleanup)

##  1. Complete prerequisites <a id ='Complete%20prerequisites'> </a>

선수 조건들(prerequisites)을 확인하고 완료해 주세요.

###  A. Check and configure access to the Internet <a id ='Check%20and%20configure%20access%20to%20the%20Internet'> </a>

이 노트북은 필요한 소프트웨어 업데이트를 다운로드하고 데이터셋을 다운로드하기 위해 인터넷에 대한 아웃바운드 액세스가 필요합니다. 다이렉트 인터넷 액세스(기본값)나 VPC를 통해 인터넷 액세스가 가능하며, 이에 대한 자세한 내용은 [여기](https://docs.aws.amazon.com/sagemaker/latest/dg/appendix-notebook-and-internet-access.html) 를 참조하세요.

###  B. Check and upgrade required software versions <a id ='Check%20and%20upgrade%20required%20software%20versions'> </a>

이 노트북은 아래 패키지들이 필요합니다.
* [SageMaker Python SDK version 2.x](https://sagemaker.readthedocs.io/en/stable/v2.html)
* [TensorFlow version 2.x with SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/tf.html)
* [Python 3.6.x](https://www.python.org/downloads/release/python-360/)
* [SMDebug](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-analyze-data.html)
* [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)

노트북을 처음 실행하는 경우, `install_needed = True`로 설정해 주세요.

In [ ]:
%load_ext autoreload
%autoreload 2

install_needed = False
#install_needed = True

In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U smdebug boto3 matplotlib bokeh seaborn
    IPython.Application.instance().kernel.do_shutdown(True)
    
import boto3
import sagemaker
import smdebug
import tensorflow as tf

# Get the current installed version of Sagemaker SDK, TensorFlow, Python, Boto3 and SMDebug
print('SageMaker Python SDK version : {}'.format(sagemaker.__version__))
print('TensorFlow version : {}'.format(tf.__version__))
print('Python version : {}'.format(sys.version))
print('Boto3 version : {}'.format(boto3.__version__))
print('SMDebug version : {}'.format(smdebug.__version__))    

###  C. Check and configure security permissions <a id ='Check%20and%20configure%20security%20permissions'> </a>
이 노트북은 기본 노트북 인스턴스에 연결된 IAM role을 사용합니다. 이 role의 이름을 보려면 다음 셀을 실행하세요.

Note: 이 role에는 다음 권한이 있어야 합니다.

1. 훈련 및 출력 데이터를 저장하는 데 사용될 S3 버킷에 대한 전체 액세스 권한
2. 훈련 인스턴스 시작에 대한 전체 액세스 권한
3. CloudWatch에 쓰기 위한 액세스 권한

SageMaker 노트북 인스턴스를 생성했고 노트북 인스턴스의 주피터 노트북으로 접속하면 `sagemaker.get_execution_role()`으로 위의 권한을 모두 충족하는 role을 가져올 수 있습니다. 하지만, 실제 프로덕션 적용 시에는 사용자별로 권한을 각자 부여하여 보안에 신경써야 합니다.

In [ ]:
role = sagemaker.get_execution_role()
print(role)

###  D. Organize imports <a id ='Organize%20imports'> </a>

SageMaker 훈련 및 배포에 필요한 기본적인 라이브러리 및 모듈을 import합니다. 

- SageMaker Python SDK: Amazon SageMaker의 기능들을 쉽게 사용할 수 있는 고수준(high-level)의 라이브러리입니다.
- boto3 SDK: HTTP API 호출을 숨기는 편한 추상화 모델을 가지고 있고, Amazon EC2 인스턴스 및 S3 버킷과 같은 AWS 리소스와 동작하는 파이선 클래스를 제공하는 저수준(low-level)의 라이브러리입니다.

In [ ]:
from IPython.core.display import display, HTML
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from datetime import date
from sagemaker.debugger import (ProfilerConfig,
                                FrameworkProfile,
                                CollectionConfig,
                                DebuggerHookConfig,
                                DetailedProfilingConfig, 
                                DataloaderProfilingConfig, 
                                PythonProfilingConfig,
                                Rule,
                                PythonProfiler,
                                cProfileTimer,
                                ProfilerRule,
                                rule_configs)
from sagemaker.tensorflow import TensorFlow
from smdebug.profiler.analysis.notebook_utils.metrics_histogram import MetricsHistogram
from smdebug.profiler.analysis.notebook_utils.step_timeline_chart import StepTimelineChart
from smdebug.profiler.analysis.notebook_utils.step_histogram import StepHistogram
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts
from smdebug.profiler.analysis.notebook_utils.heatmap import Heatmap
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

###  E. Create common objects <a id='Create%20common%20objects'></a>

이 노트북의 향후 단계에서 사용할 공통 object들을 생성합니다.

Note: `s3_bucket = sagemaker.Session().default_bucket()`은 SageMaker에서 자동으로 생성해 주는 디폴트 S3 버킷입니다. 여러분이 별도로 생성한 S3 버킷을 사용하고 싶다면, `s3_bucket`을 변경해 주세요.

In [ ]:
# Specify the S3 bucket name
#s3_bucket = '<Specify the S3 bucket name>'
s3_bucket = sagemaker.Session().default_bucket()

# Create the S3 Boto3 resource
s3_resource = boto3.resource('s3')
s3_bucket_resource = s3_resource.Bucket(s3_bucket)

# Get the AWS region name
region_name = sagemaker.Session().boto_region_name

today = date.today()
str_date = today.strftime("%Y-%m-%d")

# Base name to be used to create resources
nb_name = f'tf2-fashion-mnist-custom-sdp-debugger-{str_date}'

# Names of various resources
train_job_name = 'train-{}'.format(nb_name)

# Names of local sub-directories in the notebook file system
data_dir = os.path.join(os.getcwd(), 'data/{}'.format(nb_name))
train_dir = os.path.join(os.getcwd(), 'data/{}/train'.format(nb_name))
test_dir = os.path.join(os.getcwd(), 'data/{}/test'.format(nb_name))

# Sub-folder names in S3
train_dir_s3_prefix = '{}/data/train'.format(nb_name)
test_dir_s3_prefix = '{}/data/test'.format(nb_name)

# Location in S3 where the training scripts will be copied
code_location = 's3://{}/{}/scripts'.format(s3_bucket, nb_name)

# Location in S3 where the model checkpoint will be stored
model_checkpoint_s3_path = 's3://{}/{}/checkpoint/'.format(s3_bucket, nb_name)

# Location in S3 where the trained model and debugger output will be stored
model_and_debugger_output_s3_path = 's3://{}/{}/output/'.format(s3_bucket, nb_name)

## 2. Prepare the dataset <a id ='Prepare%20the%20dataset'> </a>

[Fashion MNIST 데이터셋](https://github.com/zalandoresearch/fashion-mnist)은 10개의 패션 카테고리의 60,000장의 28x28 그레이스케일 이미지로 구성된 훈련 데이터셋과 10,000장의 이미지로 구성된 테스트셋으로 구성됩니다. 클래스 범주는 0에서 9까지의 정수로 매핑되며 다음 클래스 레이블을 나타냅니다.

* 0: T-shirt/top
* 1: Trouser
* 2: Pullover
* 3: Dress
* 4: Coat
* 5: Sandal
* 6: Shirt
* 7: Sneaker
* 8: Bag
* 9: Ankle boot

### A) Create the local directories <a id='Create%20the%20local%20directories'></a>

데이터셋이 복사되고 처리될 로컬 시스템에 디렉토리를 생성합니다.

In [ ]:
# Create the local directories
os.makedirs(data_dir, exist_ok=True)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

### B) Load the dataset <a id ='Load%20the%20dataset'> </a>

keras.datasets API를 사용하여 사전 셔플된 훈련 및 테스트 데이터를 로드합니다.

In [ ]:
# Load the dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

### C) View the details of the dataset <a id ='View%20the%20details%20of%20the%20dataset'> </a>


In [ ]:
# Summarize the dataset
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

### D) Visualize the dataset <a id ='Visualize%20the%20dataset'> </a>

테스트 데이터셋에서 `sample_size` 개수만큼의 이미지와 라벨을 무작위로 표시합니다.

In [ ]:
# Randomly display the images and labels of n (sample_size) images from the test dataset

sample_size = 50

random_indexes = np.random.randint(0, len(x_test), sample_size)
sample_images = x_test[random_indexes]
sample_labels = y_test[random_indexes]
sample_predictions = None
num_rows = 5
num_cols = 10
plot_title = None
fig_size = None
assert sample_images.shape[0] == num_rows * num_cols

# Labels
FASHION_LABELS = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}

import seaborn as sns

with sns.axes_style("whitegrid"):
    sns.set_context("notebook", font_scale=1.1)
    sns.set_style({"font.sans-serif": ["Verdana", "Arial", "Calibri", "DejaVu Sans"]})
    f, ax = plt.subplots(num_rows, num_cols, figsize=((14, 9) if fig_size is None else fig_size),
        gridspec_kw={"wspace": 0.02, "hspace": 0.30}, squeeze=True)
    for r in range(num_rows):
        for c in range(num_cols):
            image_index = r * num_cols + c
            ax[r, c].axis("off")
            ax[r, c].imshow(sample_images[image_index], cmap="Greys")
            if sample_predictions is None:
                title = ax[r, c].set_title("%s" % FASHION_LABELS[sample_labels[image_index]])
            else:
                true_label = sample_labels[image_index]
                pred_label = sample_predictions[image_index]
                prediction_matches_true = (sample_labels[image_index] == sample_predictions[image_index])
                if prediction_matches_true:
                    title = FASHION_LABELS[true_label]
                    title_color = 'g'
                else:
                    title = '%s/%s' % (FASHION_LABELS[true_label], FASHION_LABELS[pred_label])
                    title_color = 'r'
                title = ax[r, c].set_title(title)
                plt.setp(title, color=title_color)
    if plot_title is not None:
        f.suptitle(plot_title)
    plt.show()
    plt.close()

이미지의 픽셀 값은 0~255 범위이며, 아래에서 확인할 수 있습니다.

In [ ]:
plt.figure()
plt.imshow(x_train[0])
plt.colorbar()
plt.grid(False)
plt.show()

### E) Normalize the dataset <a id ='Normalize%20the%20dataset'> </a>

픽셀 값의 범위는 0에서 255까지이므로 0에서 1 사이의 범위로 정규화합니다. 본 예제에서는 단순하게 255로 나누어 수행하며, 이는 훈련 및 테스트 이미지 모두에 대해 수행되어야 합니다.

Note: 모델에 따라서 z-normalization, minmax scaling, [-1, 1] 사이로 정규화할 수 있습니다.

In [ ]:
# Normalize the dataset
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

### F) Save the prepared datasets locally <a id='Save%20the%20prepared%20datasets%20locally'></a>

준비된 훈련 데이터셋과 테스트 데이터셋을 로컬 디렉터리에 저장합니다. 저장하기 전에 필요에 따라 x 및 y 열을 연결합니다. 존재하지 않는 경우 디렉터리를 생성해 주세요.

In [ ]:
# Save the prepared dataset (in numpy format) to the local directories
np.save(os.path.join(train_dir, 'x_train.npy'), x_train)
np.save(os.path.join(train_dir, 'y_train.npy'), y_train)
np.save(os.path.join(test_dir, 'x_test.npy'), x_test)
np.save(os.path.join(test_dir, 'y_test.npy'), y_test)

### G) Upload the prepared datasets to S3 <a id ='Upload%20the%20prepared%20datasets%20to%20S3'> </a>

로컬 디렉터리의 데이터셋을 지정된 S3 버킷의 적절한 하위 디렉터리로 업로드합니다.

In [ ]:
# Upload the data to S3
train_data_s3_full_path = sagemaker.Session().upload_data(path='./data/{}/train/'.format(nb_name),
                                                          bucket=s3_bucket,
                                                          key_prefix=train_dir_s3_prefix)
test_data_s3_full_path = sagemaker.Session().upload_data(path='./data/{}/test/'.format(nb_name),
                                                         bucket=s3_bucket,
                                                         key_prefix=test_dir_s3_prefix)

## 3. View the training script <a id ='View%20the%20training%20script'> </a>

모델 훈련에 사용될 스크립트를 확인합니다. 이 파일은 로컬 디렉토리에 있어야 합니다. 

### A) Zero script change <a id ='Zero%20script%20change'> </a>

SageMaker 디버거 후크를 사용하기 위해 훈련 스크립트를 수정할 필요가 없는 경우입니다.

In [ ]:
!pygmentize scripts/train_tf2_fashion_mnist_custom_sdp.py

### B) With script change <a id ='With%20script%20change'> </a>

SageMaker 디버거 후크를 사용하도록 훈련 스크립트를 수정해야 하는 경우(예를 들어 필요에 따라 학습 스크립트의 특정 지점에 스칼라 및 텐서를 저장하려면) 아래 코드를 참조하세요.

In [ ]:
!pygmentize scripts/train_tf2_fashion_mnist_custom_sdp_debugger.py

##  4. Perform training, validation and testing <a id ='Perform%20training%20validation%20and%20testing'> </a>

이 단계에서는 SageMaker에 최적화된 TensorFlow 2.x 컨테이너를 사용하여 SageMaker Distributed Data Parallel 라이브러리 및 사용자 지정 훈련 루프를 사용하는 [Fashion MNIST 데이터셋](https://github.com/zalandoresearch/fashion-mnist) 을 사용하여 다중 클래스 이미지 분류 모델의 분산 훈련을 수행합니다.

디버거는 훈련 프로세스의 일부로 활성화됩니다. 디버거 설정에 따라 필요한 수의 [Processing Jobs](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html)이 생성됩니다.

Note:

* 사용자 지정 훈련 루프 및 SageMaker Distributed Data Parallel 라이브러리에 대한 로직은 훈련 스크립트에 있습니다.
* 훈련 프로세스 중에 모델을 체크 포인트하는 것을 권장드리며, 특히 spot 인스턴스를 사용할 때는 훈련 작업이 중단될 가능성이 있으므로 적극 권장합니다.

### A) Set the training parameters <a id ='Set%20the%20training%20parameters'> </a>

1. Inputs - 훈련 및 테스트 데이터를 위한 S3 위치.
2. 하이퍼파라메터 및 체크 포인트 파라메터
3. SageMaker Distributed Data Parallel distribution 파라메터
4. 훈련 인스턴스 세부 정보:

    1. 인스턴스 개수 (Note: 최소 2개 필요)
    2. 인스턴스 타입 (Note: SageMaker Distributed Data Parallel이 지원하는 인스턴스 유형은 [여기](https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel-faq.html) 를 참조하세요.    
    3. Training job의 max run time    
    4. (Optional) 스팟 인스턴스 사용. 자세한 내용은 [여기](https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html)를 참조하세요.
    5. (Optional) 스팟 인스턴스의 max wait time (스팟 인스턴스 사용 시에만). 이 파라메터는 max run time보다 커야 합니다.
    6. 훈련 이미지 파라메터. 자세한 내용은 [여기](https://github.com/aws/deep-learning-containers/blob/master/available_images.md)를 참조하세요.


5. 기본 작업 이름(Base job name), Tensorflow 프레임워크 버전 및 Python 버전.
6. 훈련 스크립트의 이름과 스크립트가 있는 로컬 디렉터리.
7. SageMaker에 최적화된 Tensorflow 2.x 컨테이너의 로깅 수준.
8. 훈련 작업에서 사용할 적절한 로컬 및 S3 디렉터리.

In [ ]:
# Set the input data paths
inputs = {'train':train_data_s3_full_path, 'test':test_data_s3_full_path}

# Location where the model checkpoints will be stored locally in the container before being uploaded to S3
## Note: It is recommended that you use the default location of /opt/ml/checkpoints/ for saving/loading checkpoints.
model_checkpoint_local_dir = '/opt/ml/checkpoints/'

# Set the hyperparameters
## Note: 'batch_size' indicates the total size of all the batches that will be processed by every GPU
## used in this training process.  For example, a ml.p3.16xlarge instance has 8 GPUs.  If you specify
## an instance count of 2, then you will get 16 GPUs to train on.  In this case, if the 'batch_size' 
## is 1600, then SageMaker Distributed Data Parallel library will distribute it in such a way that
## every GPU will get a batch size of 100.
##
##
## Note: Parameters 'checkpoint_enabled', 'checkpoint_load_previous' and 'checkpoint_local_dir' are not
## hyperparameters.  They have been specified here as a means to pass them to the training script.
## The better way of passing these would be in the Environment variables which were not supported
## at the time of writing this notebook.
##
## 'checkpoint_enabled' - when this is set to 'True', the training script will save the model as a checkpoint
## after every epoch.  If set to 'False', checkpoints will not be saved.
##
## 'checkpoint_load_previous' - when this is set to 'True', prior checkpoints saved to S3 will be downloaded
## to the container and the weights from the latest checkpoint from that list will be loaded to the model.
## Training will resume from that point.  If this is set to 'False', prior checkpoints saved to S3 will still
## be downloaded to the container but not loaded for training.  In this case, training will start from scratch.
##
## 'checkpoint_local_dir' - the local directory in the container where the checkpoints will be saved to
## and loaded from.
hyperparameters = {'epochs': 25,
                   'batch_size': 1600,
                   'learning_rate': 0.001,
                   'decay': 1e-6,
                   'checkpoint_enabled': 'True',
                   'checkpoint_load_previous': 'True',
                   'checkpoint_local_dir': model_checkpoint_local_dir}

# Set the distribution
distribution = { "smdistributed":
                { "dataparallel":
                 { "enabled": True
                 }
                }
               } 

# Set the instance count, instance type, instance volume size, options to use Spot instances and other parameters
## Required minimum instance count of 2
train_instance_count = 2
## Supported instance types are 'ml.p3.16xlarge', 'ml.p3dn.24xlarge', 'ml.p4d.24xlarge', 'local_gpu'
train_instance_type = 'ml.p3.16xlarge'
train_instance_volume_size_in_gb = 30
#use_spot_instances = True
#spot_max_wait_time_in_seconds = 5400
use_spot_instances = False
spot_max_wait_time_in_seconds = None
max_run_time_in_seconds = 3600

# Training image parameters
framework_version = '2.4.1'
py_version = 'py37'
cuda_version = 'cu110'
image_os_version = 'ubuntu18.04'
image_uri = '763104351884.dkr.ecr.{}.amazonaws.com/tensorflow-training:{}-gpu-{}-{}-{}'.format(region_name,
                                                                                               framework_version,
                                                                                               py_version,
                                                                                               cuda_version,
                                                                                               image_os_version)

# Set the training script related parameters
train_script_dir = 'scripts'
## Zero-script-change scenario
train_script = 'train_tf2_fashion_mnist_custom_sdp.py'
## With-script-change scenario
#train_script = 'train_tf2_fashion_mnist_custom_sdp_debugger.py'

# Set the training container related parameters
container_log_level = logging.INFO

# Location where the trained model will be stored locally in the container before being uploaded to S3
model_local_dir = '/opt/ml/model'

### B) Set the debugger parameters <a id ='Set%20the%20debugger%20parameters'> </a>

1. **프로필(Profile) 설정** - 훈련 작업에서 시스템 지표(metric) 및 프레임워크 지표를 수집하고 보안 S3 버킷 URI 또는 로컬 머신에 저장하는 방법을 구성합니다.

    1. [Monitoring hardware system resource utilization](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-configure-system-monitoring.html)
    2. [Framework profiling](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-configure-framework-profiling.html)
 

2. **디버거 후크(Debugger hook) 설정** - 훈련 작업에서 출력 텐서를 수집하고 보안 S3 버킷 URI 또는 로컬 머신에 저장하는 방법을 구성합니다. 자세한 내용은 여기를 참조하십시오.

3. **규칙(Rules)** - 병렬로 실행하려는 디버거 기본 제공 규칙을 사용하려면 이 파라메터를 설정하세요. 규칙은 자동으로 훈련 작업을 분석하고 훈련 문제를 찾습니다. ProfilerReport 규칙은 보안 S3 버킷 URI에 디버거 프로파일링 보고서를 저장합니다. 자세한 내용은 [여기](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-configure-hook.html) 를 참조하세요.

In [ ]:
# Location in S3 where the debugger output will be stored is mentioned in the previous step

# Set the profile config for both system and framework metrics
profiler_config = ProfilerConfig(
    system_monitor_interval_millis = 500,
    framework_profile_params = FrameworkProfile(
        detailed_profiling_config = DetailedProfilingConfig(
            start_step = 5, 
            num_steps = 10
        ),
        dataloader_profiling_config = DataloaderProfilingConfig(
            start_step = 7, 
            num_steps = 10
        ),
        python_profiling_config = PythonProfilingConfig(
            start_step = 9, 
            num_steps = 10,
            python_profiler = PythonProfiler.CPROFILE, 
            cprofile_timer = cProfileTimer.TOTAL_TIME
        )
    )
)

# Set the debugger hook config to save tensors
debugger_hook_config = DebuggerHookConfig(
    collection_configs = [
        CollectionConfig(name = 'weights'),
        CollectionConfig(name = 'gradients')
    ]
)

# Set the rules to analyze tensors emitted during training
## These specific set of rules will inspect the overall training performance and progress of the model
rules = [
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.stalled_training_rule())
]

### C) (Optional) Delete previous checkpoints <a id='(Optional)%20Delete%20previous%20checkpoints'></a>

이전에 훈련했던 모델의 체크포인트가 이전 스텝에서 지정된 S3 체크포인트 위치에서 발견되면 자동으로 훈련 프로세스를 실행하는 컨테이너에 다운로드됩니다. 체크포인트 매개 변수 `checkpoint_load_previous`를 `True`로 설정하면 최신 체크포인트 파일의 가중치가 모델에 로드되고 여기에서 훈련이 시작됩니다. `checkpoint_load_previous`를 False로 설정한 경우, 해당 체크포인트 파일을 S3에서 삭제하여 S3에서 컨테이너로 불필요한 체크포인트 파일 다운로드를 방지할 수 있습니다. 이를 위해 다음 코드 셀을 실행하십시오.

In [ ]:
# Delete the checkpoints if you want to train from the beginning; else ignore this code cell
for checkpoint_file in s3_bucket_resource.objects.filter(Prefix='{}/checkpoint/'.format(nb_name)):
    checkpoint_file_key = checkpoint_file.key
    print('Deleting {} ...'.format(checkpoint_file_key))
    s3_resource.Object(s3_bucket_resource.name, checkpoint_file_key).delete()

### D) Run the training job <a id='Run%20the%20training%20job'></a>

역주: `estimator`를 준비하고 `fit(training_data_uri)` 메서드를 호출합니다. 그러면 AWS 리전에서 지정된 버전의 프레임워크가 포함된 컨테이너를 가져와서 사용자 지정 훈련 스크립트를 여기에 복사하고 지정된 유형의 EC2 인스턴스에서 훈련 작업을 실행합니다. 훈련 데이터는 S3의 지정된 위치에서 가져오며, 실제 컨테이너 환경에서는 SageMaker 환경 변수인 `SM_CHANNEL_TRAINING` 경로로 복사됩니다. 훈련 종료 후 최종 생성된 모델은 체크포인트와 함께 S3의 지정된 위치에 기록됩니다. 디버거는 구성된 설정을 사용하여 데이터를 캡처하고 S3의 지정된 위치에 기록합니다.

In [ ]:
# Create the estimator
estimator = TensorFlow(
    source_dir=train_script_dir,
    entry_point=train_script,
    code_location=code_location,
    #checkpoint_local_path=model_checkpoint_local_dir,
    checkpoint_s3_uri=model_checkpoint_s3_path,
    model_dir=model_local_dir,
    output_path=model_and_debugger_output_s3_path,
    distribution=distribution,
    instance_type=train_instance_type,
    volume_size=train_instance_volume_size_in_gb,
    instance_count=train_instance_count,
    use_spot_instances=use_spot_instances,
    max_wait=spot_max_wait_time_in_seconds,
    max_run=max_run_time_in_seconds,
    hyperparameters=hyperparameters,
    role=role,
    base_job_name=train_job_name,
    image_uri=image_uri,
    container_log_level=container_log_level,
    disable_profiler=False,
    profiler_config=profiler_config,
    debugger_hook_config=debugger_hook_config,
    rules=rules)

#### Perform the training

역주: `fit()`에서 `wait=True`로 설정할 경우 Synchronous 방식으로 동직하게 되며, `wait=False`일 경우 Aynchronous 방식으로 동작되어 여러 개의 훈련 job을 동시에 실행할 수 있습니다.

In [ ]:
estimator.fit(inputs, wait=False)

`wait=False`로 설정한 경우 아래 코드 셀을 실행하여 Synchronous 방식으로 변경할 수도 있습니다.

In [ ]:
sagemaker.Session().logs_for_job(job_name=estimator.latest_training_job.job_name, wait=True)

## 5. View the auto-generated debugger profiling report <a id ='View%20the%20auto-generated%20debugger%20profiling%20report'> </a>

디버거의 자동 생성 프로파일링 리포트는 이전 단계에서 지정한 S3 디렉터리에 저장됩니다.
리포트를 읽는 방법에 대한 정보는 [여기](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-profiling-report.html)를 참조하세요.

In [ ]:
# # Get the S3 path to the debugger's auto-generated profiling report
# profiling_report_s3_prefix = '{}/output/{}/rule-output/ProfilerReport/profiler-output/profiler-report.html'.format(nb_name,
#                                                                              estimator.latest_training_job.job_name)
# profiling_report = sagemaker.Session().read_s3_file(s3_bucket, profiling_report_s3_prefix)

# # Print debugger's auto-generated profiling report location
# display(HTML(profiling_report))

In [ ]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

In [ ]:
output_dir = './output'
profile_output = output_dir+'/ProfilerReport'

!rm -rf $output_dir

import json, os

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
if not os.path.exists(profile_output):
    os.makedirs(profile_output)    

In [ ]:
!aws s3 ls {rule_output_path}/ProfilerReport/profiler-output/

In [ ]:
!aws s3 cp {rule_output_path}/ProfilerReport/profiler-output/ {output_dir}/ProfilerReport/ --recursive

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>ProfilerReport : <a href="{}profiler-report.html">Profiler Report</a></b>'.format(output_dir+"/ProfilerReport/")))

## 6. Perform interactive analysis of the debugger output <a id ='Perform%20interactive%20analysis%20of%20the%20debugger%20output'> </a>


디버거의 출력은 이전 단계에서 지정한 S3 디렉터리에 저장됩니다. 이 단계에서는 해당 데이터를 읽고 다양한 시각화를 통해 표시합니다.

### A. Get the training job <a id ='Get%20the%20training%20job'> </a>

이전 단계에서 훈련에 사용된 estimator 객체에서 훈련 job 객체(object)를 가져옵니다. 이는 디버거의 출력에서 메트릭을 읽는 데 필요합니다.

In [ ]:
# This assumes that the job was trained in the same AWS region as the S3 bucket where the debugger output is stored
# If not, then make appropriate changes to the following code
tj = TrainingJob(estimator.latest_training_job.job_name, sagemaker.Session().boto_region_name)

### B. Read the metrics <a id ='Read%20the%20metrics'> </a>

1. 시스템 및 프레임워크 메트릭을 사용할 수 있을 때까지 기다립니다.
2. 이 두 메트릭에 대한 reader 객체를 가져옵니다.
3. 이러한 메트릭이 포함된 이벤트 파일 목록을 갱신합니다.

In [ ]:
# Wait for the data to be available
tj.wait_for_sys_profiling_data_to_be_available()
tj.wait_for_framework_profiling_data_to_be_available()
# Get the metrics reader
system_metrics_reader = tj.get_systems_metrics_reader()
framework_metrics_reader = tj.get_framework_metrics_reader()
# Refresh the event file list
system_metrics_reader.refresh_event_file_list()
framework_metrics_reader.refresh_event_file_list()

### C. Plot the metrics <a id ='Plot%20the%20metrics'> </a>

이전 단계에서 읽은 메트릭에 대한 시각화를 플로팅(plotting)합니다.

#### a. System metrics histogram <a id ='System%20metrics%20histogram'> </a>

In [ ]:
metrics_histogram = MetricsHistogram(system_metrics_reader)
metrics_histogram.plot(
    starttime=0, 
    endtime=system_metrics_reader.get_timestamp_of_latest_available_file(), 
    select_dimensions=["CPU", "GPU", "I/O"],
    select_events=["total"]
)

#### b. Framework metrics stepline chart <a id ='Framework%20metrics%20stepline%20chart'> </a>

In [ ]:
view_step_timeline_chart = StepTimelineChart(framework_metrics_reader)

#### c. Framework metrics step histogram <a id ='Framework%20metrics%20step%20histogram'> </a>

In [ ]:
step_histogram = StepHistogram(framework_metrics_reader)
step_histogram.plot(
    starttime=step_histogram.last_timestamp - 5 * 1000 * 1000, 
    endtime=step_histogram.last_timestamp, 
    show_workers=True
)

#### d. System and framework metrics timeline charts <a id ='System%20and%20framework%20metrics%20timeline%20charts'> </a>

In [ ]:
view_timeline_charts = TimelineCharts(
    system_metrics_reader, 
    framework_metrics_reader,
    select_dimensions=["CPU", "GPU", "I/O"],
    select_events=["total"] 
)

view_timeline_charts.plot_detailed_profiler_data([700,710]) 

#### e. System and framework metrics heatmap <a id ='System%20and%20framework%20metrics%20heatmap'> </a>

In [ ]:
view_heatmap = Heatmap(
    system_metrics_reader,
    framework_metrics_reader,
    select_dimensions=["CPU", "GPU", "I/O"],
    select_events=["total"],
    plot_height=450
)

## 7. Cleanup <a id='Cleanup'></a>

더 이상 필요하지 않은 경우 리소스 및 S3 객체를 삭제해야 합니다. 이렇게 하면 불필요한 비용이 발생하는 것을 방지할 수 있습니다.

아래 코드 셀은 이 노트북에서 생성된 리소스와 S3 객체를 정리합니다.

In [ ]:
# Delete data from S3 bucket
for file in s3_bucket_resource.objects.filter(Prefix='{}/'.format(nb_name)):
    file_key = file.key
    print('Deleting {} ...'.format(file_key))
    s3_resource.Object(s3_bucket_resource.name, file_key).delete()